In [ ]:
import os, sys 
#to be able to interact with Google Drive's operating system
from google.colab import drive 
#drive is a module that allows us use Python to interact with google drive
drive.mount('/content/gdrive') 
#mounting google drive allows us to work with its contents
nb_path = '/content/notebooks'
os.symlink('/content/gdrive/My Drive/Colab Notebooks', nb_path)
sys.path.insert(0, nb_path)  # or append(nb_path)
#The last three lines are what changes the path of the file.c

Mounted at /content/gdrive


Install the required package

In [ ]:
!pip install tigramite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.3/197.3 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 75.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 73.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 43.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.39.1
    Uninstalling llvmlite-0.39.1:
      Successfully uninstalled llvmlite-0.39.1
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.3
    Uninstalling scipy-1.7.3:
      Successfully uninstalled scipy-1.7.3
  Attempting uninstall: numba
    Fo

Import requied libraries

In [ ]:
# Imports
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     
## use `%matplotlib notebook` for interactive figures
# plt.style.use('ggplot')
#import sklearn
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

/usr/local/lib/python3.8/dist-packages/tigramite/plotting.py:26: UserWarning: [Errno 2] No such file or directory: '/usr/local/lib/python3.8/dist-packages/tigramite/../versions.py'
  warnings.warn(str(e))
/usr/local/lib/python3.8/dist-packages/tigramite/independence_tests/gpdc.py:27: UserWarning: No module named 'dcor'
  warnings.warn(str(e))
/usr/local/lib/python3.8/dist-packages/tigramite/independence_tests/gpdc_torch.py:33: UserWarning: No module named 'dcor'
  warnings.warn(str(e))


Upload the dataset we are interested to find the lagged causal links

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 6.2.csv to 6.2.csv


In [ ]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['6.2.csv']))# Dataset is now stored in a Pandas Dataframe

In [ ]:
df=df.to_numpy()
print(df)

[[0.80461592 1.33724272 1.99459802 1.44945304 2.92115399 1.71136555]
 [1.10133242 0.83854238 2.402612   2.14784303 2.08612208 0.9023248 ]
 [1.24397146 1.76475677 3.19862332 2.07038714 2.48033943 0.75063101]
 ...
 [1.2312914  2.49156927 7.51045166 3.80706239 5.97925753 8.3338776 ]
 [0.97959564 2.02923289 7.09642714 3.92715775 5.38262862 9.18684169]
 [1.04292525 2.04021347 7.68191971 4.74105042 5.83325179 9.30615662]]


In [ ]:
# Initialize dataframe object, specify time axis and variable names
var_names = [r'$X^1$', r'$X^2$', r'$X^3$', r'$X^4$', r'$X^5$', r'$X^6$']
dataframe = pp.DataFrame(df, 
                         datatime = np.arange(len(df)), 
                         var_names=var_names)

Perform the test using the parameters: significance level of the CI tests, minimum and maximum period of lags. Record the lagged links for the further steps.

In [ ]:
cond_ind_test = CMIknn()
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)
results = pcmci.run_pcmciplus(tau_min=0, tau_max=2, pc_alpha=0.05)
pcmci.print_results(results, alpha_level=0.05)


## Significant links at alpha = 0.05:

    Variable $X^1$ has 1 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.072

    Variable $X^2$ has 2 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.035
        ($X^2$ -2): pval = 0.00600 | val =  0.020

    Variable $X^3$ has 2 link(s):
        ($X^3$ -2): pval = 0.00000 | val =  0.018
        ($X^2$ -2): pval = 0.00000 | val =  0.015

    Variable $X^4$ has 1 link(s):
        ($X^3$  0): pval = 0.00000 | val =  0.036

    Variable $X^5$ has 3 link(s):
        ($X^4$ -2): pval = 0.00000 | val =  0.025
        ($X^6$  0): pval = 0.00900 | val =  0.023 | unoriented link
        ($X^5$ -2): pval = 0.00000 | val =  0.021

    Variable $X^6$ has 2 link(s):
        ($X^5$  0): pval = 0.00900 | val =  0.023 | unoriented link
        ($X^6$ -2): pval = 0.02600 | val =  0.017

## Ambiguous triples (not used for orientation):

    [($X^3$ -2), $X^3$, $X^4$]
    [($X^6$ -2), $X^6$, $X^5$]
    [($X^4$ -2), $X^5$, $X^6$]
    [($X^5$ -2), $X^5

133 mins

In [ ]:
cond_ind_test = CMIknn()
pcmci = PCMCI(dataframe=dataframe, cond_ind_test=cond_ind_test)
results = pcmci.run_pcmciplus(tau_min=1, tau_max=2, pc_alpha=0.05)
pcmci.print_results(results, alpha_level=0.05)


## Significant links at alpha = 0.05:

    Variable $X^1$ has 1 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.072

    Variable $X^2$ has 2 link(s):
        ($X^1$ -1): pval = 0.00000 | val =  0.035
        ($X^2$ -2): pval = 0.00500 | val =  0.020

    Variable $X^3$ has 2 link(s):
        ($X^3$ -2): pval = 0.00000 | val =  0.024
        ($X^2$ -2): pval = 0.00000 | val =  0.021

    Variable $X^4$ has 1 link(s):
        ($X^2$ -2): pval = 0.00000 | val =  0.021

    Variable $X^5$ has 2 link(s):
        ($X^4$ -2): pval = 0.00000 | val =  0.027
        ($X^5$ -2): pval = 0.00200 | val =  0.021

    Variable $X^6$ has 1 link(s):
        ($X^6$ -2): pval = 0.02300 | val =  0.019


57.5 mins